## Exceptions
Exceptions are error conditions defined by a type. Ex. division by zero.
IndexError, KeyError, NameError, ZeroDivisionError, etc.

How to trap exceptions with try-except loops:

`try:
    try this code here
except KeyError:
    if it doesn't work, use this code here
    `
    
Can also do tuples of all possible errors: `except(ValueError,IndexError)`. When getting an error, python will output a Traceback, aka a _stack trace_. It tells you at what point in each function or method call the error occurred. 

In [3]:
#Example:

class MyClass(object):
    @staticmethod
    def make_error():
        print('entering make_error()')
        #5/0  let's comment this out to remove the error
        print('   leaving make_error()')
        
    def do_something(self):
        print('entering do_something()')
        self.make_error()
        print('   leaving do_something()')
        
def some_func():
    print('entering some_func()')
    cc = MyClass()
    cc.do_something()
    print('   leaving some_func()')
def major_func():
    print('entering major_func()')
    some_func()
    print('   leaving major_func()')
def main():
    print('entering main()')
    major_func()
    print('   leaving main()')
    
main()

entering main()
entering major_func()
entering some_func()
entering do_something()
entering make_error()
   leaving make_error()
   leaving do_something()
   leaving some_func()
   leaving major_func()
   leaving main()


The error exception can be placed anywhere within the call stack.

How do we raise an exception ourselves? May want to raise a different exception than python, or if we think something is wrong but python does not. 

In [5]:
def make_delim_line(list_to_join,delim):
    try:
        formatted_line = delim.join(list_to_join)
    except TypeError:
        raise TypeError('make_delim_line(): arg 1 must be a list or tuple')
    
    return formatted_line

fline = make_delim_line(100,',')

TypeError: make_delim_line(): arg 1 must be a list or tuple

Instances of the exception class becomes available to us at the time that we trap it, if we follow the class name (i.e. TypeError). Built-in python exceptions can be found [here](https://docs.python.org/2/library/exceptions.html).

In [15]:
mydict = {'a':1,'b':2}

try:
    print mydict[c]
except NameError as e:
    print(e.args)

("name 'c' is not defined",)


### Define our own Exception Types

We can create our own exception classes and inherit from parent classes. Usually they don't do much more than display a message so it's easy for other users to know what's going on. It's important to trap and handle user errors because they're the most common type of error. 

In [19]:
class MyError(Exception):
    def __init__(self,*args):
        print 'calling init'
        if args:
            self.message = args[0]
        else:
            self.message = ''
    def __str__(self):
        print 'calling str'
        if self.message:
            return "here's a MyError exception with a message: {}".format(self.message)
        else:
            return "here's a MyError exception"
        
#raise MyError
raise MyError('Houston, we have a problem!')

calling init
calling str


MyError: here's a MyError exception with a message: Houston, we have a problem!

calling str
calling str


## Assignment 4 Testing

Think about our class from assignment 3 and any user errors that might occur and how to handle them. Create a custom error class that handles the dictionary 'KeyError' by listing out the keys that _do_ exist within the dictionary. 

Things that could go wrong:
* file doesn't exist / path to file is bad?
* ask for a key that doesn't exist


add a `__getitem__(self,key)` to ConfigDict

In [24]:
import os

class ConfigDict(dict):

    def __init__(self,filename):
        self._filename=filename
        if os.path.isfile(self._filename):
            with open(self._filename) as fh:
                for line in fh:
                    line = line.rstrip()
                    key,val = line.split("=",1)
                    dict.__setitem__(self,key,val)

    def __setitem__(self,key,val):
        dict.__setitem__(self,key,val)
        with open(self._filename,'w') as fh:
            for key,val in self.items():
                fh.write("{}={}\n".format(key,val))
                
    def __getitem__(self,key):
        try:
            return dict.__getitem__(self,key)
        except KeyError:
            raise ConfigKeyError(self,key)
            
class ConfigKeyError(Exception):
    def __init__(self,*args):
        self.message = "Key '{}' not found. Available keys: {}".format(args[1],tuple(args[0].keys()))
        
    def __str__(self):
        return self.message